# QG-tagger using REP

# Folding Strategy
REP implements folding strategy as one more metaestimator.

When we don't have enough data to split data on train/test, we're stick to k-folding cross-validation scheme. Folding becomes the only way when you use some multi-staged stacking algorithm.

Usually we split training data into folds manually, but this is annoying and not reliable. REP has FoldingClassifier and FoldingRegressor, which do this automatically.

- https://github.com/yandex/rep/blob/master/howto/04-howto-folding.ipynb
- https://github.com/yandex/rep/blob/master/howto/03-howto-gridsearch.ipynb

In [ ]:
%%time 
import numpy
import pandas
import pandas as pd
from rep.utils import train_test_split
import sklearn
from sklearn.metrics import roc_auc_score
from collections import OrderedDict

# Import from root_pandas library
from root_pandas import read_root

print sklearn.__version__
print pandas.__version__

## Loading data
Upload Monte Carlo Data Set from DESY

In [ ]:
%%time
## Data loading function

def load(sig_filename, bkg_filename, category, features):
    """load fucntion.

    Parameters
    ----------
    sig_filename : array, shape = [n_samples]
            true class, intergers in [0, n_classes - 1)
    bkg_filename : array, shape = [n_samples, n_classes]
    category: string
    features: array, shape = [n_features]

    Returns
    -------
    data : pandas.DataFrame
    """

    # Read in ROOT file and produce panda dataframes
    signal = read_root([sig_filename], category, 
                       columns=features+['noexpand:pt_dr_log/pt'], 
                       where= 'partonId < 4 && axis2 < 8')
    #'partonId!=21 && axis2 < 8 && jetIdLevel==3 && matchedJet==1 && nGenJetsInCone==1 && nGenJetsForGenParticle==1 && nJetsForGenParticle==1 && partonId < 4 && balanced==1 && charged_multiplicity >= 3 && charged_multiplicity <= 143'
    signal['y']= 1 # add target column for signal

    background = read_root([bkg_filename], category, 
                           columns=features+['noexpand:pt_dr_log/pt'],
                           where='partonId==21 && axis2 < 8')
    #'partonId==21 && axis2 < 8 && jetIdLevel==3 && matchedJet==1 && nGenJetsInCone==1 && nGenJetsForGenParticle==1 && nJetsForGenParticle==1 && balanced==1 && charged_multiplicity >= 3 && charged_multiplicity <= 143'
    background['y']= 0 # add target column for background

    data = pd.concat([signal, background])
    
    return data                       

In [ ]:
%%time
## Load input data files

# Feature names
branch_names = """axis1,axis2,ptD,charged_multiplicity,pt,pt_dr_log,partonId,jetIdLevel,matchedJet,nGenJetsInCone,nGenJetsForGenParticle,nJetsForGenParticle,balanced,weight""".split(",")

features = [c.strip() for c in branch_names]
features = (b.replace(" ", "_") for b in features)
features = list(b.replace("-", "_") for b in features)

# Delcare dataset location
signal_sample     = "QGtagger_training/pt_bin10_eta_bin1.root"
background_sample = "QGtagger_training/pt_bin10_eta_bin1.root"
tree_category = "tree"

# Load the data to panda dataframe object
data = load(signal_sample, background_sample, tree_category, features)

print "Total number of events: {}\nNumber of features: {}".format(len(data.index), len(data.columns))

## Training variables

In [ ]:
%%time 
labels = data.y
print data.columns
variables = list(data.drop(['pt','partonId','jetIdLevel','matchedJet',
                           'nGenJetsInCone','nGenJetsForGenParticle',
                           'nJetsForGenParticle','nJetsForGenParticle',
                           'pt_dr_log','balanced','y'], axis=1, inplace=False).columns)
print "Candidate features", variables

In [ ]:
%%time 
#train_data, test_data, train_labels, test_labels = train_test_split(data.drop(["weight","y"], axis=1, inplace=False), labels, train_size=0.5)
train_data, test_data, train_labels, test_labels = train_test_split(data[variables], labels, train_size=0.5)
print train_data.shape
print test_data.shape
print train_labels.shape
print test_labels.shape

## Folding strategy

FoldingClassifier implements the same interface as all classifiers, but with some difference:
- prediction methods have additional parameter "vote_function" (example folder.predict(X, vote_function=None)), which is used to combine all classifiers' predictions.

In [ ]:
%%time 
from rep.estimators import SklearnClassifier
from sklearn.ensemble import GradientBoostingClassifier
from rep.metaml import FoldingClassifier

## Define folding model

In [ ]:
%%time
n_folds = 4
folder = FoldingClassifier(GradientBoostingClassifier(n_estimators=30), 
                           n_folds=n_folds, features=filter(lambda feature: feature!='weight', variables),
                           parallel_profile='threads-4')

folder.fit(train_data.drop(["weight"], axis=1, inplace=False), train_labels, sample_weight=train_data.weight)

## Default prediction (predict ith fold by ith classifier)
In this case each sample will be predict by estimator that was not using this particular sample in training.

When you apply this prediction to some new data (not the same was passed in training), it will predict each sample by random estimator.

In [ ]:
%%time 
prob = folder.predict_proba(train_data) 
print 'ROC AUC:', roc_auc_score(train_labels, prob[:, 1], sample_weight=train_data.weight)

In [ ]:
%%time 
prob = folder.predict_proba(test_data)
print 'ROC AUC:', roc_auc_score(test_labels, prob[:, 1], sample_weight=test_data.weight)

## Voting prediction 
(predict ith fold by all classifiers and take value, which is calculated by vote_function)
It makes sense to use all classifier to predict new data, because averaging makes predictions more stable.


In [ ]:
%%time 
# definition of mean function, which combines all predictions
def mean_vote(x):
    return numpy.mean(x, axis=0)

In [ ]:
%%time
prob = folder.predict_proba(test_data, vote_function=mean_vote)
print 'ROC AUC:', roc_auc_score(test_labels, prob[:, 1], sample_weight=test_data.weight)

# Comparison of folds
Again use ClassificationReport class to compare different results. For folding classifier this report uses only default prediction.

## Report training dataset

In [ ]:
%%time 
from rep.data.storage import LabeledDataStorage
from rep.report import ClassificationReport

# add folds_column to dataset to use mask
train_data["FOLDS"] = folder._get_folds_column(len(train_data))
lds_train = LabeledDataStorage(data=train_data.drop(["weight"], axis=1, inplace=False), 
                               target=train_labels, sample_weight=train_data.weight)

report = ClassificationReport({'folding': folder}, lds_train)

# Signal distribution for each fold

Use mask parameter to plot distribution for the specific fold

In [ ]:
%%time 
%pylab inline
for fold_num in range(n_folds):
    report.prediction_pdf(mask="FOLDS == %d" % fold_num, labels_dict={1: 'sig fold %d' % fold_num}).plot()

# Background distribution for each fold¶

In [ ]:
%%time 
%pylab inline
for fold_num in range(n_folds):
    report.prediction_pdf(mask="FOLDS == %d" % fold_num, labels_dict={0: 'bck fold %d' % fold_num}).plot()

## ROCs (each fold used as test dataset)

In [ ]:
%%time 
%pylab inline
for fold_num in range(n_folds):
    report.roc(mask="FOLDS == %d" % fold_num).plot()

## Report for test dataset
*NOTE*: Here vote function is None, so default prediction is used

In [ ]:
%%time 
# add folds_column to dataset to use mask
lds_test = LabeledDataStorage(data=test_data.drop(["weight"], axis=1, inplace=False), target=test_labels, sample_weight=test_data.weight)

report = ClassificationReport({'folding': folder}, lds_test)

In [ ]:
%%time 
%pylab inline
report.prediction_pdf().plot(new_plot=True, figsize = (9, 4))

In [ ]:
%%time 
# Scatter plot
scatter(data.axis2, data.ptD, alpha = 0.1, color='b', label='signal')
scatter(data.axis2, data.ptD, alpha = 0.01, color='r', label='background')
xlabel('pt_dr_log/pt', fontsize=16)
ylabel('ptD', fontsize=16)
title('Correlation plot', fontsize=16)

In [ ]:
%%time 
report.roc().plot(xlim=(0., 1.))

In [ ]:
%%time 
from rep.report.metrics import RocAuc
from rep.metaml import GridOptimalSearchCV, FoldingScorer, RandomParameterOptimizer
from rep.estimators import SklearnClassifier, TMVAClassifier, XGBoostRegressor
print TMVAClassifier.__doc__

In [ ]:
%%time 
# define grid parameters
grid_param = {}
grid_param['learning_rate'] = [0.2, 0.1, 0.05, 0.02, 0.01]
grid_param['max_depth'] = [2, 3, 4]
grid_param['n_estimators'] = [100, 200]

# use random hyperparameter optimization algorithm 
generator = RandomParameterOptimizer(grid_param, n_evaluations=10)

# define folding scorer
scorer = FoldingScorer(RocAuc(), folds=3, fold_checks=3)

In [ ]:
%%time 
estimator = SklearnClassifier(GradientBoostingClassifier(n_estimators=30))
grid_finder = GridOptimalSearchCV(estimator, generator, scorer, parallel_profile='threads-4')
grid_finder.fit(data[filter(lambda feature: feature!='weight', variables)], labels, sample_weight=data['weight'])

## Looking at results

In [ ]:
%%time 
grid_finder.params_generator.print_results()

## Optimizing the parameters and threshold

In many applications we need to optimize some binary metrics for classification (f1, BER, misclassification error), in which case we need each time after training classifier to find optimal threshold on predicted probabilities (default one is usually bad).

In this example:
- We are optimizing AMS (binary metric, that was used in Higgs competition at kaggle)
- Tuning parameters of TMVA's GBDT
- Using Gaussian Processes to make good guesses about next points to check

In [ ]:
%%time 
from rep.metaml import RegressionParameterOptimizer
from sklearn.gaussian_process import GaussianProcess, GaussianProcessRegressor
from rep.report.metrics import OptimalMetric, OptimalAccuracy, OptimalAMS, OptimalSignificance, ams, OptimalSignificance

In [ ]:
%%time
# OptimalMetrics is a wrapper which is able to check all possible thresholds
# expected number of signal and background events are taken as some arbitrary numbers
optimal_ams = OptimalMetric(ams, expected_s=100, expected_b=1000)

# define grid parameters
grid_param = OrderedDict(
    {'NTrees': [1],#[1000], [5, 10, 15, 20, 25], 
     'MinNodeSize' : [2.5],
     'Shrinkage': [0.20], #[0.4, 0.2, 0.1, 0.05, 0.02, 0.01], 
     'UseBaggedBoost:BaggedSampleFraction': [0.5],
     'nCuts': [20],
     'MaxDepth': [2],
     # you can pass different sets of features to be compared
     'features': [variables[:1], variables[:2]]
     #'features': [filter(lambda feature: feature!='weight', variables), variables[:1]]
     #'features': [variables[:2], variables[:3], variables[:4]],
    }
)

# using GaussianProcesses 
generator = RegressionParameterOptimizer(grid_param, n_evaluations=1, 
                                         regressor=GaussianProcessRegressor(), 
                                         n_attempts=1)

# define folding scorer
scorer = FoldingScorer(optimal_ams, folds=2, fold_checks=1)
#"Silent=True:V=False:DrawProgressBar=False"
#grid_finder = GridOptimalSearchCV(TMVAClassifier(method='kBDT', BoostType='Grad', 
#                                                 factory_options="!V:!Silent:Color:DrawProgressBar:Transformations=I;D;P;G,D:AnalysisType=Classification"),
grid_finder = GridOptimalSearchCV(TMVAClassifier(method='kBDT', BoostType='Grad', 
                                                 factory_options="!V=False:!Silent=True:DrawProgressBar=False:AnalysisType=Classification"),
                                                                    
                                  generator, scorer, parallel_profile='threads-4')

In [ ]:
%%time
grid_finder.fit(data, labels, sample_weight=data.weight)

In [ ]:
grid_finder.fit(data[filter(lambda feature: feature!='weight', variables)], labels, sample_weight=data.weight)

## Looking at results

In [ ]:
grid_finder.generator.print_results()

## Let's see dynamics over time

In [ ]:
%pylab inline
plot(grid_finder.generator.grid_scores_.values())

## Optimizing complex models + using custom scorer
REP supports sklearn-way of combining classifiers and getting/setting their parameters.
So you can tune complex models using the same approach.

Let's optimize
- BaggingRegressor over XGBoost regressor, we will select appropriate parameters for both
- we will roll new scorer, which test everything on special part of dataset
- we use the same data, which will be once split into train and test (this scenario of testing is sometimes needed)
- optimizing MAE (mean absolute error)

In [ ]:
%%time
from sklearn.ensemble import BaggingRegressor
from rep.estimators import XGBoostRegressor

In [ ]:
%%time
from sklearn.metrics import mean_absolute_error
from sklearn.base import clone

class MyMAEScorer(object):
    def __init__(self, test_data, test_labels):
        self.test_data = test_data
        self.test_labels = test_labels
        
    def __call__(self, base_estimator, params, X, y, sample_weight=None):
        cl = clone(base_estimator)
        cl.set_params(**params)
        cl.fit(X, y)
        # Returning with minus, because we maximize metric
        return -mean_absolute_error(self.test_labels, cl.predict(self.test_data))

In [ ]:
%%time
# define grid parameters
grid_param = OrderedDict(
    {
    # parameters of sklearn Bagging
    'n_estimators': [1, 2], #[1, 3, 5, 7],
    'max_samples': [0.1], #[0.2, 0.4, 0.6, 0.8],
    # parameters of base (XGBoost)
    'base_estimator__n_estimators': [1, 2], #[10, 20, 40], 
    'base_estimator__eta': [0.1] #[0.1, 0.2, 0.4, 0.6, 0.8]
    }
)

# using Gaussian Processes 
generator = RegressionParameterOptimizer(grid_param, n_evaluations=4, 
                                         regressor=GaussianProcessRegressor(), 
                                         n_attempts=10)

estimator = BaggingRegressor(XGBoostRegressor(), n_estimators=10)

scorer = MyMAEScorer(test_data, test_labels)
##scorer(sample_weight=data["weight"]) #does not work!!

In [ ]:
%%time
grid_finder = GridOptimalSearchCV(estimator, generator, scorer, parallel_profile='threads-4')
grid_finder.fit(data[filter(lambda feature: feature!='weight', variables)], labels, sample_weight=data['weight'])

In [ ]:
%%time 
grid_finder.generator.print_results()

In [ ]:
## The following two secions below are still in developement. Neeed to figure out if ipyton can handle threading

In [ ]:
%%time
from sklearn.model_selection import KFold, StratifiedKFold, ShuffleSplit

## Standard nested k-fold cross-validation
def nestedGridSearchCV(Classifier,
                       generator,
                       X, y,
                       outer_cv, 
                       param_grid, 
                       scorer, 
                       parallel_profile):
#def nestedGridSearchCV(Classifier, X, y, outer_cv, inner_cv, 
#                       parameter_grid, scoring="accuracy"):
    """Nested k-fold crossvalidation."""
    
    """ 
    Parameters
    ----------
    Classifier : array, shape = [n_samples]
            true class, intergers in [0, n_classes - 1)
    X : array, shape = [n_samples, n_classes]
    y : array, shape = [n_samples, n_classes]
    outer_cv:  shape = [n_samples, n_classes]
    inner_cv:  shape = [n_samples, n_classes]
    parameter_grid: shape = [n_samples, n_classes]
    scoring:   shape = [n_samples, n_classes]
    
    Returns
    -------
    Grid classifier: classifier re-fitted to full dataset
    """    
    
    
    outer_scores = []
    
    for training_samples, test_samples in outer_cv.split(X, y):

        # Training datasets
        x_training_temp = pd.DataFrame(X.iloc[training_samples], columns=features)

        x_training = x_training_temp.drop('weight', axis=1, inplace=False)
        y_training = pd.Series(y.iloc[training_samples])

        # Extract sample weight
        weights_training = x_training_temp["weight"].values

        # Testing datasets
        x_testing_temp = pd.DataFrame(X.iloc[test_samples], columns=features)

        x_testing = x_testing_temp.drop('weight', axis=1, inplace=False)
        y_testing = pd.Series(y.iloc[test_samples])

        # set up grid search configuration
        #cv = GridSearchCV(estimator=Classifier, param_grid=param_grid,
        #                  cv=inner_cv, scoring="accuracy", 
        #                  n_jobs=-1,
        #                  fit_params={"classifier__sample_weight": weights_training})
        cv = GridOptimalSearchCV(Classifier, generator, scorer, parallel_profile=None)
                         
        # train on the training set
        cv.fit(x_training, y_training)
        
        # evaluate
        #outer_scores.append(cv.score(x_testing, y_testing))
        print cv.generator.grid_scores_.values()
        outer_scores.append(cv.generator.grid_scores_.values())

    # Print final model evaluation (i.e. mean cross-validation scores)
    #print "Final model evaluation (mean cross-val scores):\n", np.array(outer_scores).mean()
    
    # note: the scoring is being done without the weights associated with X
    # fit model to entire training dataset (i.e tuning & validation dataset)
    #cv.best_estimator_.fit(X.drop('weight', axis=1, inplace=False), y,
    #                       **{"classifier__sample_weight": X["weight"].values})
    
    
    #return cv

In [ ]:
%%time
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(data, labels)

grid_param = {
    # parameters of sklearn Bagging
    'n_estimators': [1, 2], 
    'max_samples': [0.1],
    # parameters of base (XGBoost)
    'base_estimator__n_estimators': [1, 2], 
    'base_estimator__eta': [0.1]
}

k_fold=5
outer_kfold_cv = KFold(n_splits=k_fold, shuffle=True, random_state=42)

# using Gaussian Processes 
generator = RegressionParameterOptimizer(grid_param, n_evaluations=4, 
                                         regressor=GaussianProcessRegressor(), 
                                         n_attempts=10)

estimator = BaggingRegressor(XGBoostRegressor(), n_estimators=10)

scorer = MyMAEScorer(test_data, test_labels)
##scorer(sample_weight=data["globalTimesEventWeight"]) #does not work!!

grid_finder = nestedGridSearchCV(Classifier=estimator,
                                 generator=generator,
                                 X=train_data, y=train_labels,
                                 outer_cv=outer_kfold_cv, 
                                 param_grid=grid_param, 
                                 scorer=scorer, 
                                 parallel_profile='threads-3')

# Meta-ML: Factories

- http://www.programcreek.com/python/example/86675/sklearn.metrics.roc_

In [ ]:
%%time
from rep.metaml import ClassifiersFactory
from rep.estimators import TMVAClassifier, SklearnClassifier, XGBoostClassifier
from sklearn.ensemble import AdaBoostClassifier

grid_param = OrderedDict({"MaxDepth": [4, 5], "NTrees": [10, 20]})

# Define classifiers
factory = ClassifiersFactory()

#There are different ways to add classifiers to Factory
factory.add_classifier('tmva', TMVAClassifier(NTrees=15, features=filter(lambda feature: feature!='weight', variables), Shrinkage=0.1, factory_options="Silent=True:V=False:DrawProgressBar=False"))
factory.add_classifier('ada', AdaBoostClassifier(n_estimators=10))

#tmva = TMVAClassifier(method='kBDT', NTrees=15, Shrinkage=0.1, nCuts=-1, BoostType='Grad', features=filter(lambda feature: feature!='weight', variables))
#ada = AdaBoostClassifier(n_estimators=100)
#factory.add_classifier('tmva', tmva) 
#factory.add_classifier('ada', ada)

factory['xgb'] = XGBoostClassifier(features=filter(lambda feature: feature!='weight', variables)) # training

#factory.fit(train_data, train_labels, features=variables, parallel_profile='IPC')
factory.fit(train_data, train_labels, features=filter(lambda feature: feature!='weight', variables), parallel_profile='threads-4')

# predict
#factory.predict_proba(some_data, parallel_profile='IPC')
factory.predict_proba(test_data, parallel_profile='threads-4')

In [ ]:
# define metric functions
def significance(s, b):
    br = 0.01
    radicand = s/numpy.sqrt(b+br)
    return radicand

In [ ]:
from rep.report.metrics import significance
metrics = report.metrics_vs_cut(significance, metric_label='significance')
metrics.plot(new_plot=True, figsize=(10,4))

In [ ]:
# user define metric
def AMS(s,b):
    b_reg = 0.01
    radicand = 2*((s+b+b_reg) * numpy.log(1.0 + s/(b+b_reg)) -s)
    return numpy.sqrt(radicand)

metrics =report.metrics_vs_cut(AMS, metric_label='ams')
metrics.plot(new_plot=True, figsize=(10,4))

In [ ]:
report = factory.test_on(test_data, test_labels) 
learning_curve = report.learning_curve(RocAuc(), metric_label='ROC AUC', steps=10) 
learning_curve.plot()

In [ ]:
## Plot data information: features correlation matrix
report.features_correlation_matrix_by_class(features=filter(lambda feature: feature!='weight', variables)).plot(new_plot=True, show_legend=False, figsize=(15,5))

In [ ]:
# Plot distributions for each feature
# use just common features for all classifiers
report.features_pdf().plot()

In [ ]:
# Feature importance
report.feature_importance().plot()

In [ ]:
# Get features importance using shuffling method 
# (apply random permutation to one particular column)
report.feature_importance_shuffling().plot()

In [ ]:
report = factory.test_on(test_data, test_labels)

In [ ]:
# Roc curves
report.roc().plot(xlim=(0, 1))

In [ ]:
%%time
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import RobustScaler

estimator = Pipeline([('feature_scaling', None), 
                 ('feature_selection', None), 
                 ('classifier', DummyClassifier())]
               )

In [ ]:
%%time
# feature selection
select = VarianceThreshold()

# create classifier for use in scikit-learn
model = GradientBoostingClassifier()

# preprocessing using 0-1 scaling by removing the mean and scaling to unit variance 
scaler = RobustScaler()

# prepare models: create a mapping of ML classifier name to algorithm
param_grid = [
    {'classifier': [model],
     'classifier__n_estimators': [10, 20, 30, 40],
     'classifier__max_depth': [3, 4, 5],
     'classifier__learning_rate': [0.01, 0.1, 0.4],
     'feature_selection': [select],
     'feature_selection__threshold': [0.5],
     'feature_scaling': [scaler]
    }
]


print estimator.get_params().keys()

# use random hyperparameter optimization algorithm 
generator = RandomParameterOptimizer(grid_param, n_evaluations=3)

# define folding scorer
scorer = FoldingScorer(RocAuc(), folds=3, fold_checks=3)

grid_finder = GridOptimalSearchCV(estimator, generator, scorer, parallel_profile='threads-4')
grid_finder.fit(data[filter(lambda feature: feature!='weight', variables)], labels, sample_weight=data.weight)

# Summary
Grid search in REP extends sklearn grid search, uses optimization techniques to avoid complete search of estimator parameters.

REP has predefined scorers, metric functions, optimization techniques. Each component is replaceable and you can optimize complex models and pipelines (Folders/Bagging/Boosting and so on).

## Structure together
- ParameterOptimizer is responsible for generating new set of parameters which will be checked
 - RandomParameterOptimizer
 - AnnealingParameterOptimizer
 - SubgridParameterOptimizer
 - RegressionParameterOptimizer (this one can use any regression model, like GaussianProcesses)
- Scorer is responsible for training and evaluating metrics
 - Folding scorer (uses metrics with REP interface), uses averaging quality after kFolding
- GridOptimalSearchCV makes all of this work together and sends tasks to cluster or separate threads.